In [1]:
from PandaStation import (
    PandaStation, FindResource, AddPackagePaths, AddRgbdSensors, draw_points, draw_open3d_point_cloud, 
    create_open3d_point_cloud)



# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import open3d as o3d
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    MakeRenderEngineVtk, DepthRenderCamera, RenderCameraCore, CameraInfo, ClippingRange,  DepthImageToPointCloud,
    BaseField, RenderEngineVtkParams, ConnectMeshcatVisualizer, DepthRange, RgbdSensor, MeshcatPointCloudVisualizer,
    LoadModelDirectives, ProcessModelDirectives, GeometrySet
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

import matplotlib.pyplot as plt


In [2]:
'''
def plot_manipulation_station_camera_images():
    station = PandaStation()
    station.SetupBinStation()
    station.Finalize()
    context = station.CreateDefaultContext()

    camera_names = station.get_camera_info()
    index = 1
    for name, pose in camera_names.items():
        color_image = station.GetOutputPort(name +
                                            "_rgb_image").Eval(context)
        depth_image = station.GetOutputPort(name +
                                            "_depth_image").Eval(context)

        plt.subplot(len(camera_names), 2, index)
        plt.imshow(color_image.data)
        index += 1
        plt.title('Color image')
        plt.subplot(len(camera_names), 2, index)
        plt.imshow(np.squeeze(depth_image.data))
        index += 1
        plt.title('Depth image')

    plt.show()

plot_manipulation_station_camera_images()   

def deg_to_rad(deg):
    return deg*np.pi/180.0

X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.2]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.5, 0, 0.09])}

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupBinStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    X_O["initial"])


plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")

station.Finalize()


station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


frames_to_draw = {"cameras": "camera"}

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url,
          frames_to_draw=frames_to_draw)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions


X_Camera = pydrake.math.RigidTransform(
                 pydrake.math.RollPitchYaw(deg_to_rad(-150), 0, np.pi/2.0).ToRotationMatrix(),
                [0.3, -0.65, 1])


meshcat_pointcloud = builder.AddSystem(MeshcatPointCloudVisualizer(meshcat))#, X_WP=X_Camera))
builder.Connect(station.GetOutputPort("camera_point_cloud"), meshcat_pointcloud.get_input_port())

# Export the point cloud output.
builder.ExportOutput(station.GetOutputPort("camera_point_cloud"),
                         "point_cloud")


diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation

t = 0

meshcat.start_recording()

simulator.AdvanceTo(10)

meshcat.stop_recording()
meshcat.publish_recording()
'''
print()

In [3]:
directive = FindResource("models/two_bins_w_cameras.yaml")

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
parser = Parser(plant)
AddPackagePaths(parser)
ProcessModelDirectives(LoadModelDirectives(directive), plant, parser)

plant.Finalize()
AddRgbdSensors(builder, plant, scene_graph)


meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment")

diagram = builder.Build()
context = diagram.CreateDefaultContext()

meshcat.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6013...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
Connected to meshcat-server.


In [4]:
def fix_collisions(plant, scene_graph, scene_graph_context):
    
    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    collision_pairs = query_object.ComputePointPairPenetration()
    
    for pair in collision_pairs:
        sA = GeometrySet([pair.id_A])
        sB = GeometrySet([pair.id_B])
        scene_graph.ExcludeCollisionsBetween(scene_graph_context, sA, sB)
    '''
    hand_ids = []
    hand_ids += plant.GetCollisionGeometriesForBody(plant.GetBodyByName("panda_hand"))
    hand_set = GeometrySet(hand_ids)
    for i in hand_ids:
        print('hand_id', i)
    scene_graph.ExcludeCollisionsWithin(scene_graph_context, hand_set)
    
    left_ids = []
    left_ids += plant.GetCollisionGeometriesForBody(plant.GetBodyByName("panda_hand"))
    left_set = GeometrySet(left_ids)
    for i in left_ids:
        print('left_id', i)
    scene_graph.ExcludeCollisionsWithin(scene_graph_context, left_set)
    
    right_ids = []
    right_ids += plant.GetCollisionGeometriesForBody(plant.GetBodyByName("panda_hand"))
    right_set = GeometrySet(right_ids)
    for i in left_ids:
        print('right_id', i)
    scene_graph.ExcludeCollisionsWithin(scene_graph_context, right_set)
    '''

def grasp_candidate_cost(plant_context, cloud, plant, scene_graph, scene_graph_context, adjust_X_G=False, textbox=None, meshcat=None):
    
                                                                                                                                                              
    
    body = plant.GetBodyByName("panda_hand")
    X_G = plant.GetFreeBodyPose(plant_context, body)

    # Transform cloud into gripper frame
    X_GW = X_G.inverse()
    pts = np.asarray(cloud.points).T
    p_GC = X_GW.multiply(pts)

    # Crop to a region inside of the finger box.
    crop_min = [-0.009, -0.035, 0.06]
    crop_max = [0.009, 0.035, 0.115]
    indices = np.all((crop_min[0] <= p_GC[0,:], p_GC[0,:] <= crop_max[0],
                      crop_min[1] <= p_GC[1,:], p_GC[1,:] <= crop_max[1],
                      crop_min[2] <= p_GC[2,:], p_GC[2,:] <= crop_max[2]), 
                     axis=0)
    
    
    if meshcat:
        print('drawing')
        draw_points(meshcat["points"], pts[:, indices], [1., 0, 0], size=0.01)

    if adjust_X_G and np.sum(indices)>0:
        p_GC_x = p_GC[0, indices]
        p_Gcenter_x = (p_GC_x.min() + p_GC_x.max())/2.0
        X_G.set_translation(X_G.translation() + X_G.rotation().multiply([p_Gcenter_x, 0, 0]))
        plant.SetFreeBodyPose(plant_context, body, X_G)
        X_GW = X_G.inverse()

    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    collision_pairs = query_object.ComputePointPairPenetration()
    
    #for pair in collision_pairs:
        #print('collision_pairs', pair.id_A, pair.id_B)
    # Check collisions between the gripper and the sink
    if query_object.HasCollisions():
        cost = np.inf
        print('gripper is colliding with bin')
        if textbox:
            textbox.value = "Gripper is colliding with the sink!\n"
            textbox.value += f"cost: {cost}"
        return cost

    # Check collisions between the gripper and the point cloud
    margin = 0.0  # must be smaller than the margin used in the point cloud preprocessing.
    for pt in cloud.points:
        distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=margin)
        #print(distances)
        if distances:
            cost = np.inf
            print('gripper is colliding with point cloud')
            if textbox:
                textbox.value = "Gripper is colliding with the point cloud!\n"
                textbox.value += f"cost: {cost}"
            return cost


    n_GC = X_GW.rotation().multiply(np.asarray(cloud.normals)[indices,:].T)

    # Penalize deviation of the gripper from vertical.
    # weight * -dot([0, 0, -1], R_G * [0, 1, 0]) = weight * R_G[2,1]
    cost = 20.0*X_G.rotation().matrix()[2, 1]
    print('after vertical penlization', cost)

    # Reward sum |dot product of normals with gripper x|^2
    cost -= np.sum(n_GC[0,:]**2)

    if textbox:
        textbox.value = f"cost: {cost}\n"
        textbox.value += "normal terms:" + str(n_GC[0,:]**2)
    return cost


def process_point_cloud(diagram, context, cameras, bin_name):
    """A "no frills" version of the example above, that returns the down-sampled point cloud"""
    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    # Compute crop box.
    bin_instance = plant.GetModelInstanceByName(bin_name)
    bin_body = plant.GetBodyByName("bin_base", bin_instance)
    X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
    margin = 0.001  # only because simulation is perfect!
    a = X_B.multiply([-.22+0.025+margin, -.29+0.025+margin, 0.015+margin])
    b = X_B.multiply([.22-0.1-margin, .29-0.025-margin, 2.0])
    crop_min = np.minimum(a,b)
    crop_max = np.maximum(a,b)

    # Evaluate the camera output ports to get the images.
    merged_pcd = o3d.geometry.PointCloud()
    for c in cameras:
        point_cloud = diagram.GetOutputPort(f"{c}_point_cloud").Eval(context)
        pcd = create_open3d_point_cloud(point_cloud)

        # Crop to region of interest.
        pcd = pcd.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=crop_min,
                                                max_bound=crop_max))    

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(c)
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd.orient_normals_towards_camera_location(X_C.translation())
        
        # Merge point clouds.
        merged_pcd += pcd

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    return merged_pcd.voxel_down_sample(voxel_size=0.005)

def make_environment_model(directive=None, draw=False, rng=None, num_ycb_objects=0, bin_name="bin0"):
    # Make one model of the environment, but the robot only gets to see the sensor outputs.
    if not directive:
        directive = FindResource("models/two_bins_w_cameras.yaml")

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
    parser = Parser(plant)
    AddPackagePaths(parser)
    ProcessModelDirectives(LoadModelDirectives(directive), plant, parser)

    brick = parser.AddModelFromFile(FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf"), 'brick')
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('base_link', brick), RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.09]))
    
    plant.Finalize()
    AddRgbdSensors(builder, plant, scene_graph)

    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment")

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    if draw:
        meshcat.load()
        diagram.Publish(context)


    return diagram, context

In [5]:

def generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, rng, meshcat=None):
    """
    Picks a random point in the cloud, and aligns the robot finger with the normal of that pixel. 
    The rotation around the normal axis is drawn from a uniform distribution over [min_roll, max_roll].
    """
    body = plant.GetBodyByName("panda_hand")

    index = rng.integers(0,len(cloud.points)-1)

    # Use S for sample point/frame.
    p_WS = np.asarray(cloud.points[index])
    n_WS = np.asarray(cloud.normals[index])

    if meshcat:
        vertices = np.empty((3,2))
        vertices[:, 0] = p_WS
        vertices[:, 1] = p_WS + 0.05*n_WS
        meshcat.set_object(g.LineSegments(g.PointsGeometry(vertices),
                            g.MeshBasicMaterial(color=0xff0000)))
  

    if not np.isclose(np.linalg.norm(n_WS), 1.0):
        return np.inf, None

    Gy = n_WS # gripper y axis aligns with normal
    # make orthonormal z axis, aligned with world down
    z = np.array([0.0, 0.0, -1.0])
    if np.abs(np.dot(z,Gy)) < 1e-6:
        # normal was pointing straight down.  reject this sample.
        print('here')
        return np.inf, None

    Gz = z - np.dot(z,Gy)*Gy
    Gx = np.cross(Gy, Gz)
    R_WG = RotationMatrix(np.vstack((Gx, Gy, Gz)).T)
    p_GS_G = [0, 0.035, 0.11]

    # Try orientations from the center out
    min_pitch=-np.pi/3.0
    max_pitch=np.pi/3.0
    alpha = np.array([0.5, 0.65, 0.35, 0.8, 0.2, 1.0, 0.0])
    thetas = [0]
    for theta in (min_pitch + (max_pitch - min_pitch)*alpha):
        # Rotate the object in the hand by a random rotation (around the normal).
        R_WG2 = R_WG.multiply(RotationMatrix.MakeYRotation(theta))

        # Use G for gripper frame.
        p_SG_W = - R_WG2.multiply(p_GS_G)
        p_WG = p_WS + p_SG_W 

        X_G = RigidTransform(R_WG2, p_WG)
        plant.SetFreeBodyPose(plant_context, body, X_G)
        cost = grasp_candidate_cost(plant_context, cloud, plant, scene_graph, scene_graph_context, adjust_X_G=True, meshcat=meshcat)
        X_G = plant.GetFreeBodyPose(plant_context, body)
        if np.isfinite(cost):
            return cost, X_G

        #draw_grasp_candidate(X_G)

    return np.inf, None

def draw_grasp_candidate(X_G, prefix='gripper', draw_frames=True):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    gripper = parser.AddModelFromFile(FindResource(
        "models/welded_panda_hand.urdf"), "gripper")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_hand"), X_G)
    plant.Finalize()
    
    frames_to_draw = {"gripper": {"panda_hand"}} if draw_frames else {}
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix=prefix, delete_prefix_on_load=False, frames_to_draw=frames_to_draw,  role = Role.kProximity)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    meshcat.load()
    diagram.Publish(context)

def sample_grasps_example():
    rng = np.random.default_rng()

    environment, environment_context = make_environment_model(rng=rng, num_ycb_objects=5, draw=False)

    # Another diagram for the objects the robot "knows about": gripper, cameras, bins.  Think of this as the model in the robot's head.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    AddPackagePaths(parser)
    ProcessModelDirectives(LoadModelDirectives(FindResource("models/clutter_planning.yaml")), plant, parser)
    plant.Finalize()
    
    
    v = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="planning")
    v.load()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)
    v.vis["planning/10"].set_property('visible', False)  # Hide this particular gripper

    cloud = process_point_cloud(environment, environment_context, ["camera0", "camera1", "camera2"], "bin0")
    draw_open3d_point_cloud(v.vis["cloud"], cloud, size=0.003)

    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)

    costs = []
    X_Gs = []
    fix_collisions(plant, scene_graph, scene_graph_context)
    for i in range(100):
        cost, X_G = generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, rng)#, meshcat=v.vis["sample"])
        print(cost)
        if np.isfinite(cost):      
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:1]
    for i in indices:
        draw_grasp_candidate(X_Gs[i], prefix=f"{i}th best", draw_frames=False)


sample_grasps_example()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6013...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
Connected to meshcat-server.
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6013...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
Connected to meshcat-server.
inf
inf
inf
inf
inf
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
inf
inf
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
inf
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is col

gripper is colliding with point cloud
inf
after vertical penlization -0.015347915076101087
-0.015508052800183058
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
inf
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
gripper is colliding with bin
inf
inf
after vertical penlization -0.007006454129014248
-0.00788080423204067
inf
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
gripper is colliding with point cloud
inf
Connecting to meshcat-server at